In [2]:
#CODE TO GET PERCENTAGE CHANGES BY PROGRAM
import pandas as pd
import os

In [3]:
#FEB 2024 CBO SPENDING PROJECTIONS
currentyr = 2024
years = [i for i in range(currentyr, currentyr+11)] 
cbo_ot = pd.read_excel('data/cbo-2024-02-Spending-Projections.xlsx', header=7, sheet_name=1)
cbo_ot = cbo_ot[[col for col in cbo_ot.columns if col not in years]] #remove the BA cols
cbo_ot.columns = cbo_ot.columns.str.replace('.1', '')
cbo_ot = cbo_ot.drop(columns=['Unnamed: 9', "Unnamed: 21", "Off-Budget?"]).drop(index=0).drop(cbo_ot.index[-2:])
cbo_ot['25-34'] = cbo_ot.loc[:, '2025':'2034'].sum(axis=1)

#see if this helps improve the match
cbo_ot["Agency"] = cbo_ot["Agency"].str.strip()
cbo_ot["Bureau"] = cbo_ot["Bureau"].str.strip()
cbo_ot['Title'] = cbo_ot['Title'].str.strip()

cbo_ot["Agency-Bureau-Program"] = cbo_ot["Agency"] + "BUREAU: " + cbo_ot["Bureau"] + "PROGRAM: " + cbo_ot["Title"]

byprogram_24 = cbo_ot[["Agency-Bureau-Program", "25-34"]].groupby("Agency-Bureau-Program").sum()
byprogram_24.shape

(1643, 1)

In [4]:
#FEB 2021 CBO SPENDING PROJECTIONS
cbo21_ot = pd.read_excel('data/cbo-2021-02-11-spendingprojections.xlsx', header=7, sheet_name=1)
years_21 = [i for i in range(2021, 2032)]
cbo21_ot = cbo21_ot[[col for col in cbo21_ot.columns if col not in years_21]].drop(columns=["Off-Budget?", "Unnamed: 9", "Unnamed: 21"]) #remove the BA cols, unnamed and unnecessary columns
cbo21_ot.columns = cbo21_ot.columns.str.replace('.1', '') #clean OT columns
cbo21_ot["22-31"] = cbo21_ot.loc[:, '2022':'2031'].sum(axis=1) #sum the years

#see if this helps the match
cbo21_ot["Agency"] = cbo21_ot["Agency"].str.strip()
cbo21_ot["Bureau"] = cbo21_ot["Bureau"].str.strip()
cbo21_ot['Title'] = cbo21_ot['Title'].str.strip()

cbo21_ot["Agency-Bureau-Program"] = cbo21_ot["Agency"] + "BUREAU: " + cbo21_ot["Bureau"] + "PROGRAM: " + cbo21_ot['Title'] #create a new column for agency-bureau-program

byprogram_21 = cbo21_ot[["Agency-Bureau-Program", "22-31"]].groupby("Agency-Bureau-Program").sum() #group by agency-bureau-program and sum the total
byprogram_21.shape

(1595, 1)

In [ ]:
i = 0
for program24 in byprogram_24.index:
    if program24 not in byprogram_21.index:
        print("This 2024 program is not in 2021 programs:", program24)
        i += 1
print("Total 2024 programs not in 2021 programs:", i)

In [6]:
#merge the two dataframes
merged = pd.merge(byprogram_21, byprogram_24, on="Agency-Bureau-Program", how="inner")
#merged['percent_change'] = (merged['25-34'] - merged['22-31']) / merged['22-31'] * 100
merged['percent_change'] = ((merged['25-34'] - merged['22-31']) / abs(merged['22-31'])) * 100

merged.reset_index(inplace=True)

merged["Agency"] = merged["Agency-Bureau-Program"].str.extract(r'(.*)BUREAU:')
merged["Bureau"] = merged["Agency-Bureau-Program"].str.extract(r'BUREAU:\s(.*?)PROGRAM:')
merged['Program'] = merged["Agency-Bureau-Program"].str.extract(r'PROGRAM:\s(.*)')
merged["Agency"] = merged["Agency"].str.strip()
merged["Bureau"] = merged["Bureau"].str.strip()
merged['Program'] = merged['Program'].str.strip()

merged = merged[['Agency-Bureau-Program', 'Agency', 'Bureau', 'Program', '22-31', '25-34', 'percent_change']]
merged = merged.drop(columns=["Agency-Bureau-Program"])


x = merged.query("`22-31` < 0 | `25-34` < 0")
merged

,Agency,Bureau,Program,22-31,25-34,percent_change
0,Access Board,Access Board,Salaries and expenses,106,119.0,12.264151
1,Administrative Conference of the United States,Administrative Conference of the United States,Salaries and Expenses,35,36.0,2.857143
2,Advisory Council on Historic Preservation,Advisory Council on Historic Preservation,Salaries and Expenses,82,108.0,31.707317
3,Affordable Housing Program,Affordable Housing Program,Affordable Housing Program,3313,2506.0,-24.358587
4,Allowances,Allowances,Mandatory sequestration in function 570,0,0.0,NaN
...,...,...,...,...,...,...
1425,United States Court of Appeals for Veterans Cl...,United States Court of Appeals for Veterans Cl...,Court of Appeals for Veterans Claims Retiremen...,10,30.0,200.000000
1426,United States Court of Appeals for Veterans Cl...,United States Court of Appeals for Veterans Cl...,Salaries and Expenses,437,559.0,27.917620
1427,United States Holocaust Memorial Museum,United States Holocaust Memorial Museum,Holocaust Memorial Museum,692,734.0,6.069364
1428,United States Institute of Peace,United States Institute of Peace,United States Institute of Peace,497,619.0,24.547284


In [9]:
cbo_ot[cbo_ot['Agency']=="Securities and Exchange Commission"]

,Treasury Identification Number,Title,Discretionary or Mandatory,Major Spending Category,Agency,Bureau,Function,Subfunction,2024,2025,...,2027,2028,2029,2030,2031,2032,2033,2034,25-34,Agency-Bureau-Program
801,050-0100-0-1-376,Salaries and expenses,Discretionary,Nondefense,Securities and Exchange Commission,Securities and Exchange Commission,370.0,376.0,1.0,-22.0,...,-18.0,-18.0,-19.0,-19.0,-20.0,-21.0,-22.0,-22.0,-200.0,Securities and Exchange CommissionBUREAU: Secu...
802,050-5566-0-2-376,Securities and Exchange Commission Reserve Fund,Mandatory,Other Spending,Securities and Exchange Commission,Securities and Exchange Commission,370.0,376.0,50.0,50.0,...,50.0,50.0,50.0,50.0,50.0,53.0,50.0,50.0,503.0,Securities and Exchange CommissionBUREAU: Secu...
803,050-5567-0-2-376,Investor Protection Fund,Mandatory,Other Spending,Securities and Exchange Commission,Securities and Exchange Commission,370.0,376.0,572.0,215.0,...,234.0,253.0,215.0,234.0,234.0,253.0,234.0,234.0,2340.0,Securities and Exchange CommissionBUREAU: Secu...
1888,050-9990-8-1-809,Intrabudgetary Transactions,Mandatory,Offsetting receipts,Securities and Exchange Commission,Securities and Exchange Commission,800.0,809.0,8.0,8.0,...,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,80.0,Securities and Exchange CommissionBUREAU: Secu...


In [15]:
sheetname = "by program"
filename = "output/cbo_projection_changes.xlsx"
with pd.ExcelWriter(filename, mode='a', engine="openpyxl") as writer:
    merged.to_excel(writer, sheet_name=sheetname, index=False)

BadZipFile: File is not a zip file